<a href="https://colab.research.google.com/github/Caramel96/Samuel_BuckbyCE888/blob/main/Lab_8/Week8_MyTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/sagihaider/CE888_2021/blob/main/Lab_8/data.zip?raw=true
!unzip /content/data.zip?raw=true

--2021-05-04 13:04:30--  https://github.com/sagihaider/CE888_2021/blob/main/Lab_8/data.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/sagihaider/CE888_2021/raw/main/Lab_8/data.zip [following]
--2021-05-04 13:04:31--  https://github.com/sagihaider/CE888_2021/raw/main/Lab_8/data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip [following]
--2021-05-04 13:04:31--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting resp

In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
pip install split-folders

In [4]:
import splitfolders
splitfolders.ratio('data', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 808 files [00:00, 6599.11 files/s]


In [5]:
test_path = '/content/output/test'
train_path= '/content/output/train'
validation_path = '/content/output/val'

In [6]:
train = os.listdir(train_path)
test = os.listdir(test_path)
validation = os.listdir(validation_path)

In [7]:
humans_path = '/content/data/Humans'
cats_paths = '/content/data/cats'
dogs_path = '//content/data/dogs'
horses_path = '/content/data/horses'

humans = os.listdir(humans_path)
cats = os.listdir(cats_paths)
dogs = os.listdir(dogs_path)
horses = os.listdir(horses_path)

In [8]:
print(len(humans))
print(len(cats))
print(len(dogs))
print(len(horses))

202
202
202
202


In [9]:

image_gen = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

train = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')


val = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [10]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

img_size = [304,329,3]

In [11]:
input = Input(shape = img_size)
model = InceptionResNetV2(input_tensor=input,)

225214464/225209952 [==============================] - 2s 0us/step


In [12]:
model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 151, 164, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 151, 164, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 151, 164, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

In [13]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 151, 164, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 151, 164, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 151, 164, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [15]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [16]:
history = model.fit(train, validation_data=val, batch_size=10 ,epochs=20, callbacks=my_callbacks)

Epoch 1/20
182/182 [==============================] - 442s 2s/step - loss: 0.6081 - accuracy: 0.7879 - val_loss: 0.0722 - val_accuracy: 0.9875
Epoch 2/20
182/182 [==============================] - 425s 2s/step - loss: 0.0897 - accuracy: 0.9815 - val_loss: 0.0455 - val_accuracy: 1.0000
Epoch 3/20
182/182 [==============================] - 430s 2s/step - loss: 0.0549 - accuracy: 0.9924 - val_loss: 0.0219 - val_accuracy: 1.0000
Epoch 4/20
182/182 [==============================] - 437s 2s/step - loss: 0.0420 - accuracy: 0.9935 - val_loss: 0.0293 - val_accuracy: 0.9875
Epoch 5/20
182/182 [==============================] - 434s 2s/step - loss: 0.0376 - accuracy: 0.9928 - val_loss: 0.0260 - val_accuracy: 1.0000
Epoch 6/20
182/182 [==============================] - 431s 2s/step - loss: 0.0371 - accuracy: 0.9901 - val_loss: 0.0362 - val_accuracy: 0.9875
Epoch 7/20
182/182 [==============================] - 429s 2s/step - loss: 0.0170 - accuracy: 0.9971 - val_loss: 0.0388 - val_accuracy: 0.9875

In [17]:
model.history.history

{'accuracy': [0.9189560413360596,
  0.9780219793319702,
  0.9903846383094788,
  0.9903846383094788,
  0.9931318759918213,
  0.9903846383094788,
  0.9945054650306702,
  0.9945054650306702,
  0.9958791136741638,
  0.9917582273483276,
  0.9958791136741638,
  0.9972527623176575],
 'loss': [0.3140782415866852,
  0.08795908093452454,
  0.054108504205942154,
  0.04226775839924812,
  0.03582919389009476,
  0.03298768773674965,
  0.021991781890392303,
  0.02589986100792885,
  0.02537035197019577,
  0.02537298947572708,
  0.016858147457242012,
  0.013772704638540745],
 'val_accuracy': [0.987500011920929,
  1.0,
  1.0,
  0.987500011920929,
  1.0,
  0.987500011920929,
  0.987500011920929,
  1.0,
  1.0,
  0.987500011920929,
  1.0,
  1.0],
 'val_loss': [0.0722440630197525,
  0.045490432530641556,
  0.021929066628217697,
  0.029339786618947983,
  0.0259503535926342,
  0.03620409965515137,
  0.038818709552288055,
  0.013581727631390095,
  0.008910824544727802,
  0.031384654343128204,
  0.0106474012136